In [ ]:
import sys, pdb, gc, os, json
import numpy as np
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import tensorflow as tf

# Add the parent directory to PYTHONPATH so that we can use utils.py
sys.path.append('..')
import utils

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
np.random.seed(1337) #stay 1337 AF

In [ ]:
d = 50 #dimensionality of word vectors
data = utils.build_word_vector_matrix('../data/embeddings/5/vectors_d{}.txt'.format(d))
embeddings, labels, id_to_move, move_to_id = data

In [ ]:
def print_game(encoded_game):
    for state in encoded_game:
        print(state)
        print('-----------------------------------------------------')

def plot_model_results(history):
    
    plt.rcParams["figure.figsize"] = (12, 8)
    
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.plot(history.history['acc'], label='acc')
    plt.plot(history.history['val_acc'], label='val_acc')
    plt.legend()
    plt.show()
    
    argmax = np.argmax(history.history['val_acc']) 
    mess = 'Highest val_acc at epoch {} with value of {:.3f}'
    print(mess.format(argmax + 1, history.history['val_acc'][argmax]))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.legend()
    plt.show()
    
    mess = 'Lowest val_loss at epoch {} with value of {:.2f}'
    argmin = np.argmin(history.history['val_loss'])
    print(mess.format(argmin + 1, history.history['val_loss'][argmin]))
        
def load_data(file, split=(0.7, 0.15, 0.15)):

    def test_encoding(X, labels):
        X = np.asarray(X)
        y = np.asarray(labels)
        for i, label in enumerate(labels):
            if i < len(labels) - 1:
                if label in move_to_id:
                    vec = embeddings[move_to_id[label]]
                    assert (vec == X[i + 1][i]).all()
                else:
                    print('unknown move {}'.format(labels[i]))
                    
    
    def is_game_over_move(move):
        return move in ('0-1', '1-0', '1/2-1/2')
    
    def encode_game(moves):
        
        encoded_X = []
        labels = []

        state = np.zeros((200, d))
        labels.append(moves.pop())
        
        if len(moves) > 199:
            moves = moves[0:199]
            print(len(moves))
        
        for i, move in enumerate(moves):
            if move in move_to_id:
                vec = embeddings[move_to_id[move]]
            else:
                # empty if unknown
                vec = np.full(d, 0.0)
            state[i] = vec
        
        encoded_X.append(state)
        s = np.copy(state)
        
        while len(moves) > 0:
            labels.append(moves.pop())
            s[len(moves) - 1] = np.zeros(d)
            encoded_X.append(s)
            s = np.copy(s)
            
        labels.reverse()
        encoded_X.reverse()
        # labels and encoded_X are now symetric,
        # (i.e. each encoded_X is an encoding of
        # the state of the board after the label),
        # but what we really need is for encoded_X
        # to describe the state of the board before
        # the label. So we insert an empty board 
        # state as the first element, and pop off
        # the last.
#         encoded_X.insert(0, np.zeros((200, d)))
        encoded_X.pop(0)
        
        return encoded_X, labels
    
    def encode_moves(moves):
        
        encoded = []
        labels_array = []
        
        # find indices of all last moves in a game
        last_moves = [is_game_over_move(m) for m in moves]
        last_moves = [i for i, t in enumerate(last_moves) if t]
        
        start = 0
        i = 0
        for last_move in last_moves:
            game = moves[start:last_move + 1]
            states, labels = encode_game(game)
            labels.pop() # BUG, COME BACK AND FIX THIS
            test_encoding(states, labels)
            [encoded.append(s) for s in states]
#             encoded.append(states)
            [labels_array.append(l) for l in labels]
            start = last_move + 1
#             if i == 1000 - 1:
#                 return encoded, labels #COME BACK HERE
            i = i + 1
        return encoded, labels_array
    
    # split: train, val/dev, test
    with open(file, 'r') as f:
        moves = f.read().split()
        
    train = moves[0:int(len(moves) * split[0])]
    val = moves[len(train) + 1:len(train) + int(len(moves) * split[1])]
    test = moves[len(train) + len(val) + 1:]

    X_train, y_train = encode_moves(train[0:10000])
    X_val, y_val     = encode_moves(val[0:1500])
    X_test, y_test   = encode_moves(test[0:1000])
    return np.asarray(X_train), y_train, np.asarray(X_val), y_val, np.asarray(X_test), y_test
    
def labels_to_ints(labels):
    ints = []
    for label in labels:
        if label in move_to_id:
            ints.append(move_to_id[label])
        else:
            ints.append(-1)
    return ints


In [ ]:
X_train, y_train, X_test, y_test, _, _ = load_data('../data/test_moves.txt')

In [ ]:
# One-hot encoding
uniq_labels = list(move_to_id.values())

In [ ]:
y_train = np.asarray(labels_to_ints(y_train))
y_test  = np.asarray(labels_to_ints(y_test))

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1] * X_train.shape[2]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1] * X_test.shape[2]))

In [ ]:
y_train = to_categorical(y_train, len(uniq_labels))
y_test = to_categorical(y_test, len(uniq_labels))

In [ ]:
hyper = {
    'layers': [{
        'units': 512,
        'init': 'normal',
        'activation': 'relu',
        'dropout': 0.5
        },{
        'units': 512,
        'init': 'normal',
        'activation': 'relu',
        'dropout': 0.5

        },{
        'units': len(uniq_labels),
        'init': 'normal',
        'activation': 'softmax'
    }],
    'loss': 'categorical_crossentropy',
    'optimizer': {
        'type': 'sgd',
        'lr': 0.0001
    },
    'metrics': ['accuracy']

}

model_num = 7
model, callbacks = utils.create_model(X_train.shape[1], 
                                      len(uniq_labels), 
                                      save_dir='../data/models/{}'.format(model_num),
                                      hyper=hyper,
                                      verbose=False)
print(model.summary())

In [ ]:
# early stopping
# min_delta = the mininum change in the monitored metric before waiting $patience turns before stopping
# patience = how many consecutive epochs $val_loss change < min_delta before stopping

# callbacks.append(EarlyStopping(monitor='val_loss', 
#                                min_delta=0.001,  
#                                patience=3, 
#                                verbose=1, 
#                                mode='auto'))

In [ ]:
history = model.fit(X_train, 
                    y_train, 
                    nb_epoch=25, 
                    validation_data=(X_test, y_test), 
                    callbacks=callbacks,
                    verbose=1)

In [ ]:
plot_model_results(history)